In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import scrolledtext
import speech_recognition as sr
from transformers import pipeline
import pydub
import torch
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import io
import warnings
from collections import Counter
import re

# Suppress FutureWarnings from transformers
warnings.filterwarnings('ignore', category=FutureWarning)

# Initialize the sentiment analysis pipeline
device = 0 if torch.cuda.is_available() else -1
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=device)

# Initialize the main application window
root = tk.Tk()
root.title("Audio Sentiment Analysis Dashboard")
root.geometry("1200x900")
root.configure(bg='#f0f0f0')

def transcribe_audio(audio_path):
    """Transcribe audio to text using speech recognition."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            text = "Sorry, I could not understand the audio."
        except sr.RequestError as e:
            text = f"Error: {e}"
    return text

def analyze_sentiment(text):
    """Analyze sentiment of the transcribed text."""
    result = sentiment_pipeline(text)
    sentiment = result[0]['label']
    score = result[0]['score']
    return sentiment, score

def plot_sentiment(sentiments):
    """Plot sentiment analysis results."""
    fig, ax = plt.subplots()
    sentiment_labels = ['Positive', 'Negative', 'Neutral']
    sentiment_scores = [0, 0, 0]

    for sentiment, score in sentiments:
        if sentiment == 'POSITIVE':
            sentiment_scores[0] += score
        elif sentiment == 'NEGATIVE':
            sentiment_scores[1] += score
        else:
            sentiment_scores[2] += score

    ax.bar(sentiment_labels, sentiment_scores, color=['#4CAF50', '#F44336', '#9E9E9E'])
    ax.set_ylim(0, 1)
    ax.set_ylabel('Score')
    ax.set_title('Sentiment Analysis')

    return fig

def display_plot(fig):
    """Display matplotlib plot in tkinter window."""
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = tk.PhotoImage(data=buf.getvalue())
    plot_label.config(image=image)
    plot_label.image = image
    buf.close()

def highlight_sentiments(text):
    """Highlight words based on sentiment."""
    words = re.findall(r'\w+', text)
    positive_words = set()
    negative_words = set()

    for word in words:
        result = sentiment_pipeline(word)
        sentiment = result[0]['label']
        if sentiment == 'POSITIVE':
            positive_words.add(word)
        elif sentiment == 'NEGATIVE':
            negative_words.add(word)

    return positive_words, negative_words

def get_repeated_words(text):
    """Get repeated words in the text."""
    words = re.findall(r'\w+', text.lower())
    word_counts = Counter(words)
    repeated_words = {word: count for word, count in word_counts.items() if count > 1}
    return repeated_words

def get_sentiment_explanation(sentiment):
    """Provide explanation for the sentiment classification."""
    if sentiment == 'POSITIVE':
        return "The text is classified as positive because it contains words and phrases that express favorable sentiments or emotions."
    elif sentiment == 'NEGATIVE':
        return "The text is classified as negative because it contains words and phrases that express unfavorable sentiments or emotions."
    else:
        return "The text is classified as neutral because it contains words and phrases that are neither clearly positive nor negative."

def update_text_area(text, sentiment, score):
    """Update the text area with transcription, sentiment, and repeated words."""
    positive_words, negative_words = highlight_sentiments(text)
    repeated_words = get_repeated_words(text)

    text_area.delete(1.0, tk.END)
    text_area.insert(tk.END, "Transcription:\n")
    text_area.insert(tk.END, text + "\n\n")
    
    sentiments = [(sentiment, score)]
    fig = plot_sentiment(sentiments)
    display_plot(fig)
    
    text_area.insert(tk.END, "Sentiment Analysis:\n")
    text_area.insert(tk.END, f"Sentiment: {sentiment}\nScore: {score:.2f}\n\n")

    # Explanation of sentiment
    explanation = get_sentiment_explanation(sentiment)
    text_area.insert(tk.END, f"Explanation:\n{explanation}\n\n")
    
    text_area.insert(tk.END, "Repeated Words:\n")
    for word, count in repeated_words.items():
        text_area.insert(tk.END, f"{word}: {count}\n")
    
    # Highlight positive and negative words
    start_index = '1.0'
    for word in positive_words:
        start_index = text_area.search(word, start_index, tk.END)
        if start_index:
            end_index = f"{start_index}+{len(word)}c"
            text_area.tag_add('POSITIVE', start_index, end_index)
            start_index = end_index
    text_area.tag_configure('POSITIVE', foreground='#4CAF50')  # Green color for positive words

    start_index = '1.0'
    for word in negative_words:
        start_index = text_area.search(word, start_index, tk.END)
        if start_index:
            end_index = f"{start_index}+{len(word)}c"
            text_area.tag_add('NEGATIVE', start_index, end_index)
            start_index = end_index
    text_area.tag_configure('NEGATIVE', foreground='#F44336')  # Red color for negative words

def upload_audio():
    """Handle audio file upload, transcription, and sentiment analysis."""
    file_path = filedialog.askopenfilename(title="Select Audio File", filetypes=[("Audio Files", "*.wav")])
    if not file_path:
        return

    text = transcribe_audio(file_path)
    sentiment, score = analyze_sentiment(text)
    
    update_text_area(text, sentiment, score)

def record_audio():
    """Record audio, transcribe, and analyze sentiment."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        text_area.delete(1.0, tk.END)
        text_area.insert(tk.END, "Recording... Speak now.")
        root.update()
        audio_data = recognizer.record(source, duration=10)  # Record for 10 seconds
        text_area.delete(1.0, tk.END)
        text_area.insert(tk.END, "Processing...")
        try:
            text = recognizer.recognize_google(audio_data)
            sentiment, score = analyze_sentiment(text)
            update_text_area(text, sentiment, score)
        except sr.UnknownValueError:
            text_area.insert(tk.END, "Sorry, I could not understand the audio.")
        except sr.RequestError as e:
            text_area.insert(tk.END, f"Error: {e}")

def clear_text():
    """Clear the text area and plot."""
    text_area.delete(1.0, tk.END)
    plot_label.config(image='')

def exit_app():
    """Exit the application."""
    root.quit()

def show_help():
    """Show help information."""
    messagebox.showinfo("Help", "1. Click 'Upload Audio' to select and analyze an audio file.\n"
                              "2. Click 'Record Audio' to record audio and analyze.\n"
                              "3. Use 'Clear Text' to reset the display.\n"
                              "4. Use 'Exit' to close the application.")

def show_settings():
    """Show settings information."""
    messagebox.showinfo("Settings", "Settings options are not yet implemented.")

# UI Elements
header_frame = tk.Frame(root, bg='#2196F3', pady=10)
header_frame.pack(fill=tk.X)

button_frame = tk.Frame(root, bg='#f0f0f0')
button_frame.pack(pady=10)

upload_button = tk.Button(button_frame, text="Upload Audio", command=upload_audio, padx=20, pady=10, bg='#4CAF50', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
upload_button.pack(side=tk.LEFT, padx=5)

record_button = tk.Button(button_frame, text="Record Audio", command=record_audio, padx=20, pady=10, bg='#2196F3', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
record_button.pack(side=tk.LEFT, padx=5)

clear_button = tk.Button(button_frame, text="Clear Text", command=clear_text, padx=20, pady=10, bg='#FFC107', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
clear_button.pack(side=tk.LEFT, padx=5)

exit_button = tk.Button(button_frame, text="Exit", command=exit_app, padx=20, pady=10, bg='#f44336', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
exit_button.pack(side=tk.LEFT, padx=5)

settings_button = tk.Button(button_frame, text="Settings", command=show_settings, padx=20, pady=10, bg='#9E9E9E', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
settings_button.pack(side=tk.LEFT, padx=5)

help_button = tk.Button(button_frame, text="Help", command=show_help, padx=20, pady=10, bg='#009688', fg='black', font=('Arial', 12, 'bold'), relief=tk.RAISED)
help_button.pack(side=tk.LEFT, padx=5)

text_area_frame = tk.Frame(root, bg='#f0f0f0')
text_area_frame.pack(pady=10)

text_area = scrolledtext.ScrolledText(text_area_frame, wrap=tk.WORD, width=100, height=20, padx=10, pady=10, font=('Arial', 12), bg='#ffffff', fg='black')
text_area.pack()

plot_frame = tk.Frame(root, bg='#f0f0f0')
plot_frame.pack(pady=10)

plot_label = tk.Label(plot_frame, bg='#f0f0f0')
plot_label.pack()

status_bar = tk.Label(root, text="Ready", bg='#2196F3', fg='white', anchor='w', padx=10, font=('Arial', 12))
status_bar.pack(side=tk.BOTTOM, fill=tk.X)

# Run the application
root.mainloop()



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\aakan\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-2-ffd2d8828c89>", line 164, in upload_audio
    update_text_area(text, sentiment, score)
  File "<ipython-input-2-ffd2d8828c89>", line 139, in update_text_area
    start_index = text_area.search(word, start_index, tk.END)
  File "C:\Users\aakan\anaconda3\lib\tkinter\__init__.py", line 3821, in search
    return str(self.tk.call(tuple(args)))
_tkinter.TclError: bad text index ""
